In [1]:
import numpy as np
from scipy.signal import convolve2d
import cv2

In [2]:
X = np.arange(30., dtype=np.float32).reshape(6, 5)
W = np.array([1., 4., 7., 8., 25., 3.5, 4.1, 2.3, 5.7], dtype=np.float32).reshape(3, 3)

In [3]:
# 卷积运算
# 参考 github.com/sebgao/cTensor

# 灰度图, ndim=2
def conv2d(X, W):
    ih, iw = X.shape
    kh, kw = W.shape
    im2col_strides = (X.strides[0], X.strides[1], X.strides[0], X.strides[1])
    im2col_shape = (ih - kh + 1, iw - kw + 1, kh, kw)
    xcol = np.lib.stride_tricks.as_strided(X, im2col_shape, im2col_strides, writeable=False)
    return np.tensordot(xcol, W, axes=[[-2, -1], [0, 1]])

# 多通道, ndim=3
# (每个通道使用相同的卷积核)
def conv2dmc(X, W):
    ih, iw, _ = X.shape
    kh, kw = W.shape
    im2col_strides = (X.strides[0], X.strides[1], X.strides[0], X.strides[1], X.strides[2])
    im2col_shape = (ih - kh + 1, iw - kw + 1, kh, kw, X.shape[-1])
    xcol = np.lib.stride_tricks.as_strided(X, im2col_shape, im2col_strides, writeable=False)
    return np.tensordot(xcol, W, axes=[[-3, -2], [0, 1]])


In [4]:
convolve2d(X, W, mode='same')

array([[  34.     ,  107.     ,  155.5    ,  204.     ,  202.5    ],
       [ 228.1    ,  360.     ,  420.6    ,  481.19998,  426.30002],
       [ 450.1    ,  663.     ,  723.6    ,  784.2001 ,  663.8    ],
       [ 672.1    ,  966.     , 1026.6    , 1087.2001 ,  901.3    ],
       [ 894.1    , 1269.     , 1329.6    , 1390.2001 , 1138.7999 ],
       [ 965.1    , 1206.     , 1254.6    , 1303.2001 , 1009.3    ]],
      dtype=float32)

In [5]:
cv2.filter2D(X, -1, W[::-1, ::-1], borderType=cv2.BORDER_CONSTANT)

array([[  34. ,  107. ,  155.5,  204. ,  202.5],
       [ 228.1,  360. ,  420.6,  481.2,  426.3],
       [ 450.1,  663. ,  723.6,  784.2,  663.8],
       [ 672.1,  966. , 1026.6, 1087.2,  901.3],
       [ 894.1, 1269. , 1329.6, 1390.2, 1138.8],
       [ 965.1, 1206. , 1254.6, 1303.2, 1009.3]], dtype=float32)

In [6]:
conv2d(np.pad(X, [[1, 1], [1, 1]]), W[::-1, ::-1])

array([[  34.     ,  107.     ,  155.5    ,  204.     ,  202.5    ],
       [ 228.1    ,  360.     ,  420.59998,  481.19998,  426.3    ],
       [ 450.1    ,  663.     ,  723.60004,  784.19995,  663.80005],
       [ 672.1    ,  966.     , 1026.6001 , 1087.2001 ,  901.3    ],
       [ 894.1    , 1269.     , 1329.6001 , 1390.2001 , 1138.8    ],
       [ 965.1    , 1206.     , 1254.6001 , 1303.2001 , 1009.3    ]],
      dtype=float32)

In [7]:
X3C = np.concatenate([
    X[..., None], X[..., None] * 2, X[..., None] * 3
], axis=-1)
conv2dmc(np.pad(X3C, [[1, 1], [1, 1], [0, 0]]), W)

array([[[  49.199997,   98.399994,  147.6     ],
        [ 106.2     ,  212.4     ,  318.59998 ],
        [ 154.79999 ,  309.59998 ,  464.4     ],
        [ 203.40001 ,  406.80002 ,  610.19995 ],
        [ 177.5     ,  355.      ,  532.5     ]],

       [[ 238.7     ,  477.4     ,  716.1     ],
        [ 367.19998 ,  734.39996 , 1101.6     ],
        [ 427.80002 ,  855.60004 , 1283.3999  ],
        [ 488.4     ,  976.8     , 1465.2001  ],
        [ 393.5     ,  787.      , 1180.5     ]],

       [[ 476.2     ,  952.4     , 1428.6     ],
        [ 670.19995 , 1340.3999  , 2010.6     ],
        [ 730.8     , 1461.6     , 2192.4     ],
        [ 791.39996 , 1582.7999  , 2374.2     ],
        [ 615.5     , 1231.      , 1846.5     ]],

       [[ 713.7     , 1427.4     , 2141.1     ],
        [ 973.19995 , 1946.3999  , 2919.5999  ],
        [1033.8     , 2067.6     , 3101.4001  ],
        [1094.3999  , 2188.7998  , 3283.2     ],
        [ 837.5     , 1675.      , 2512.5     ]],

       [[ 95

In [8]:
cv2.filter2D(X3C, -1, W, borderType=cv2.BORDER_CONSTANT)

array([[[  49.199997,   98.399994,  147.59999 ],
        [ 106.200005,  212.40001 ,  318.59998 ],
        [ 154.79999 ,  309.59998 ,  464.39996 ],
        [ 203.4     ,  406.8     ,  610.19995 ],
        [ 177.5     ,  355.      ,  532.5     ]],

       [[ 238.7     ,  477.4     ,  716.1     ],
        [ 367.19998 ,  734.39996 , 1101.6     ],
        [ 427.80002 ,  855.60004 , 1283.3999  ],
        [ 488.4     ,  976.8     , 1465.2     ],
        [ 393.5     ,  787.      , 1180.5     ]],

       [[ 476.2     ,  952.4     , 1428.6     ],
        [ 670.2     , 1340.4     , 2010.6     ],
        [ 730.7999  , 1461.5999  , 2192.4001  ],
        [ 791.4     , 1582.8     , 2374.2     ],
        [ 615.5     , 1231.      , 1846.5     ]],

       [[ 713.7     , 1427.4     , 2141.1     ],
        [ 973.2     , 1946.4     , 2919.5999  ],
        [1033.7999  , 2067.5999  , 3101.4001  ],
        [1094.4     , 2188.8     , 3283.2     ],
        [ 837.5     , 1675.      , 2512.5     ]],

       [[ 95

In [9]:
np.abs(conv2dmc(np.pad(X3C, [[1, 1], [1, 1], [0, 0]]), W) - cv2.filter2D(X3C, -1, W, borderType=cv2.BORDER_CONSTANT)).max()

0.00024414062